In [18]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

import warnings
warnings.filterwarnings('ignore')

In [19]:
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

In [3]:
llm = ChatOpenAI(temperature=0.0)
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

In [4]:
conversation.predict(input="Hi, my name is Andrew")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, my name is Andrew
AI:

> Finished chain.


"Hello Andrew! It's nice to meet you. How can I assist you today?"

In [5]:
conversation.predict(input="What is my name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Andrew
AI: Hello Andrew! It's nice to meet you. How can I assist you today?
Human: What is my name?
AI:

> Finished chain.


'Your name is Andrew.'

In [6]:
print(memory.buffer)

Human: Hi, my name is Andrew
AI: Hello Andrew! It's nice to meet you. How can I assist you today?
Human: What is my name?
AI: Your name is Andrew.


In [7]:
memory.load_memory_variables({})

{'history': "Human: Hi, my name is Andrew\nAI: Hello Andrew! It's nice to meet you. How can I assist you today?\nHuman: What is my name?\nAI: Your name is Andrew."}

In [8]:
# 메모리 초기화
memory = ConversationBufferMemory()

In [9]:
memory.save_context({"input": "Hi"}, 
                    {"output": "What's up"})

In [10]:
print(memory.buffer)

Human: Hi
AI: What's up


In [11]:
memory.load_memory_variables({})

{'history': "Human: Hi\nAI: What's up"}

In [12]:
memory.save_context({"input": "Not much, just hanging"}, 
                    {"output": "Cool"})

In [13]:
memory.load_memory_variables({})

{'history': "Human: Hi\nAI: What's up\nHuman: Not much, just hanging\nAI: Cool"}

In [14]:
from langchain.memory import ConversationBufferWindowMemory

In [15]:
memory = ConversationBufferWindowMemory(k=1) 

In [16]:
memory.save_context({"input": "Hi"},
                    {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})

In [17]:
memory.load_memory_variables({})

{'history': 'Human: Not much, just hanging\nAI: Cool'}

# LangChain: Q&A over Documents

In [1]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [2]:
from langchain.chains import RetrievalQA
from langchain.document_loaders import CSVLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown

In [3]:
file = 'OutdoorClothingCatalog_1000.csv'
loader = CSVLoader(file_path=file, encoding='utf-8')

In [4]:
from langchain.indexes import VectorstoreIndexCreator
from langchain_openai import OpenAIEmbeddings

In [44]:
%pip install langchain faiss-cpu openai tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 38.0 MB/s eta 0:00:00

[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: /opt/homebrew/opt/python@3.11/bin/python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=OpenAIEmbeddings()
).from_loaders([loader])

ImportError: Could not import docarray python package. Please install it with `pip install "langchain[docarray]"`.

In [25]:
query ="Please list all your shirts with sun protection \
in a table in markdown and summarize each one."

In [26]:
from langchain_openai import OpenAI

In [31]:
llm_replacement_model = OpenAI(temperature=0, model='gpt-4o-mini')
response = index.query(query, llm=llm_replacement_model)

NameError: name 'index' is not defined

In [28]:
display(Markdown(response))

NameError: name 'response' is not defined

## LLM's on Documents
우리는 언어 모델을 사용하고 이를 많은 문서와 결합하여 활용하고 싶습니다. 하지만 중요한 문제가 있습니다. 언어 모델은 한 번에 수천 단어만 검사할 수 있습니다. 그렇다면 매우 큰 문서가 있는 경우 언어 모델이 거기에 있는 모든 것에 대한 질문에 답하도록 하려면 어떻게 해야 할까요? 여기서는 임베딩(Embedding) 과 벡터 저장소(Vector Store) 가 작동합니다. 먼저 임베딩(Embedding)에 대해 알아보겠습니다.

## Embeddings
임베딩은 텍스트 조각에 대한 숫자 표현을 만듭니다. 이 숫자 표현은 지나간 텍스트 조각의 의미를 포착합니다. 유사한 내용을 가진 텍스트 조각은 유사한 벡터를 갖습니다. 이를 통해 벡터 공간의 텍스트 조각을 비교할 수 있습니다.

아래 예에서는 세 개의 문장이 있음을 알 수 있습니다. 처음 두 개는 애완동물에 관한 것이고, 세 번째는 자동차에 관한 것입니다. 숫자 공간의 표현을 보면 애완동물에 관한 문장에 해당하는 텍스트 조각의 두 벡터를 비교할 때 매우 유사하다는 것을 알 수 있습니다. 반면에 자동차에 대해 이야기하는 것과 비교하면 전혀 유사하지 않습니다. 이를 통해 어떤 텍스트 조각이 서로 유사한지 쉽게 파악할 수 있으며, 질문에 답하기 위해 언어 모델에 전달할 때 어떤 텍스트 조각을 포함할지 생각할 때 매우 유용합니다.

## Vector Database
우리가 다룰 다음 구성 요소는 벡터 데이터베이스입니다. 벡터 데이터베이스는 이전 단계에서 생성한 이러한 벡터 표현을 저장하는 방법입니다. 이 벡터 데이터베이스를 만드는 방법은 들어오는 문서에서 나오는 텍스트 덩어리로 채우는 것입니다. 큰 문서가 수신되면 먼저 이를 더 작은 덩어리로 나눕니다. 이는 원본 문서보다 작은 텍스트 조각을 만드는 데 도움이 되며, 전체 문서를 언어 모델에 전달할 수 없기 때문에 유용합니다. 그래서 우리는 이러한 작은 덩어리를 만들고 이를 벡터 데이터베이스에 저장하기 위해서 각 청크에 대한 임베딩을 생성합니다.

인덱스를 생성하면 어떤 일이 발생합니까? 이제 이 인덱스가 있으므로 런타임 중에 이를 사용하여 들어오는 쿼리와 가장 관련성이 높은 텍스트 조각을 찾을 수 있습니다. 쿼리가 들어오면 먼저 해당 쿼리에 대한 임베딩을 만듭니다. 그런 다음 이를 벡터 데이터베이스의 모든 벡터와 비교하고 가장 유사한 n개를 선택합니다. 그런 다음 이러한 내용이 반환되고 프롬프트의 내용을 언어 모델에 전달하여 최종 답변을 얻을 수 있습니다.

### Step By Step

In [29]:
loader = CSVLoader(file_path=file, encoding='utf-8')

In [ ]:
docs = loader.load()
len(docs)

In [ ]:
docs[0]

In [ ]:
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [ ]:
embed = embeddings.embed_query("Hi my name is Jun")
print(len(embed))

In [ ]:
print(embed[:5])

In [ ]:
db = DocArrayInMemorySearch.from_documents(
    docs, 
    embeddings
)

In [ ]:
query = "Please suggest a shirt with sunblocking"

In [ ]:
docs = db.similarity_search(query)
len(docs)

In [ ]:
docs[1]

In [ ]:
from langchain_openai import ChatOpenAI

In [ ]:
llm = ChatOpenAI(temperature = 0.0)

In [ ]:
qdocs = "".join([docs[i].page_content + "/n======/n/n" for i in range(len(docs))])

In [ ]:
qdocs

In [ ]:
response = llm.invoke("document: {qdocs} /n Question: From the above document, please list all your \
shirts name correctly with sun protection in a table in markdown and summarize each one.") 

In [ ]:
display(Markdown(response.content))

## Using LangChain
그렇다면 이를 어떻게 사용하여 문서에 대한 질문에 답할 수 있을까요? 먼저 이 벡터 저장소에서 검색기(retriever)를 만들어야 합니다. 검색기(retriever)는 쿼리를 받아 문서를 반환하는 모든 메서드로 뒷받침될 수 있는 일반 인터페이스입니다. 벡터 스토어와 임베딩은 그렇게 하기 위한 방법 중 하나입니다. 물론 덜 발전된 방법도 있고 좀 더 발전된 방법도 있습니다.

In [ ]:
retriever = db.as_retriever()

In [ ]:
#!pip install lanchainhub

In [ ]:
# RAG prompt
from langchain import hub

# Loads the latest version
prompt = hub.pull("rlm/rag-prompt", api_url="https://api.hub.langchain.com")

qa_stuff = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    chain_type_kwargs={"verbose": True, "prompt": prompt},
    verbose=True
)

In [ ]:
prompt

In [ ]:
query =  "Please list all your shirts with sun protection in a table \
in markdown and summarize each one."

In [ ]:
response = qa_stuff.run(query)

In [ ]:
display(Markdown(response))